1. MLE question.  Generate some data.  Try to fit with Neg-Bin.  Show some Neg-Bin to demonstrate diversity of shapes.  Log-likelihood.  

2. Update Q to Bayesian.  Give 4 possible parameter sets (multiverse idea), with uninformative priors.  Generate 1 piece of data.  Calculate posterior.  Then generate 2nd piece of data.  Recalc posterior.  Next calc P(both data|prior) and show same final posterior.

